In [1]:
%cd A:\ENV

A:\ENV


In [2]:
from setup import make
env = make("TLMN")

In [ ]:
env.render(Agent="human", per_data=[0], level=1, max_temp_frame=100)

In [ ]:
env.render(Agent=env.bot_lv0, per_data=[0], level=1, max_temp_frame=100)
